In [2]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

# MARBERT NER
model_name = "aubmindlab/bert-base-arabertv2-ner"
tokenizer_ner = AutoTokenizer.from_pretrained(model_name)
model_ner = AutoModelForTokenClassification.from_pretrained(model_name)


OSError: asafaya/bert-base-arabic-ner is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `hf auth login` or by passing `token=<your_token>`